# Tensor Interoperability Performance Benchmarks

This notebook benchmarks the performance of the new tensor interoperability features in `audio_samples` Python bindings.

**Test Configuration:**
- Sample Rate: 44100 Hz
- Sample Type: f32
- File Durations: 0.1s, 0.5s, 1s, 2s, 5s, 10s, 30s, 60s
- Channel Configurations: Mono and Stereo

**Features Tested:**
- NumPy array protocol methods (`__array__`, `__array_function__`, `__array_ufunc__`)
- DLPack protocol methods (`__dlpack__`, `from_dlpack`)
- PyTorch tensor integration (`to_torch`, `from_torch`)

## Setup and Imports

In [ ]:
import time
import timeit
import gc
import os
import tempfile
from pathlib import Path
from typing import List, Dict, Tuple, Any

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import audio_samples as aus

# Try to import torch
try:
    import torch
    TORCH_AVAILABLE = True
    print(f"PyTorch version: {torch.__version__}")
except ImportError:
    TORCH_AVAILABLE = False
    print("PyTorch not available - skipping PyTorch benchmarks")

# Define Okabe-Ito colorblind-friendly palette
OKABE_ITO_COLORS = [
    '#E69F00',  # orange
    '#56B4E9',  # sky blue  
    '#009E73',  # bluish green
    '#F0E442',  # yellow
    '#0072B2',  # blue
    '#D55E00',  # vermillion
    '#CC79A7',  # reddish purple
    '#999999'   # gray
]

# Configure plotting with same style as existing benchmark
plt.rcParams['figure.figsize'] = [12, 9]
plt.rcParams['font.size'] = 20
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['axes.labelsize'] = 22
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['legend.fontsize'] = 18
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.fancybox'] = True
plt.rcParams['legend.shadow'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['grid.linewidth'] = 1
plt.rcParams['lines.linewidth'] = 4
plt.rcParams['lines.markersize'] = 12

# Set the color palette
sns.set_palette(OKABE_ITO_COLORS)

print(f"audio_samples version: {getattr(aus, '__version__', 'unknown')}")
print(f"numpy version: {np.__version__}")

## Configuration and Utility Functions

In [ ]:
# Test configuration - same as existing benchmark
SAMPLE_RATE = 44100
SAMPLE_TYPE = np.float32
DURATIONS = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 30.0, 60.0]  # seconds
CHANNELS = [1, 2]  # mono, stereo
BENCHMARK_ITERATIONS = 50  # Same as existing benchmark
FREQUENCY = 440.0  # Hz for sine wave test signal

# Create temporary directory for results
temp_dir = tempfile.mkdtemp(prefix="tensor_bench_")
print(f"Results will be stored in: {temp_dir}")

def calculate_throughput(file_size_mb: float, time_seconds: float) -> float:
    """Calculate throughput in MB/s."""
    return file_size_mb / time_seconds if time_seconds > 0 else 0.0

def format_time(seconds: float) -> str:
    """Format time with appropriate units."""
    if seconds >= 1.0:
        return f"{seconds:.3f}s"
    elif seconds >= 0.001:
        return f"{seconds*1000:.1f}ms"
    else:
        return f"{seconds*1000000:.1f}μs"

def get_audio_size_mb(audio) -> float:
    """Calculate audio data size in MB."""
    np_array = audio.to_numpy()
    return np_array.nbytes / (1024 * 1024)

print(f"Benchmark iterations: {BENCHMARK_ITERATIONS}")

## Test Data Generation

In [ ]:
def generate_test_audio():
    """Generate test audio files for all duration/channel combinations."""
    test_audio = {}
    
    print("Generating test audio...")
    for duration in DURATIONS:
        for channels in CHANNELS:
            # Generate test signal using audio_samples
            audio = aus.generation.sine_wave(
                frequency=FREQUENCY,
                duration_secs=duration,
                sample_rate=SAMPLE_RATE,
                amplitude=0.5
            )
            
            # Convert to stereo if needed
            if channels == 2:
                # Create stereo by duplicating mono signal
                mono_data = audio.to_numpy()
                stereo_data = np.stack([mono_data, mono_data * 0.8])  # Slight variation for stereo
                # Use from_array method instead of new_multi
                try:
                    audio = aus.AudioSamples.from_array(stereo_data, sample_rate=SAMPLE_RATE)
                except:
                    # Fallback: create manually if from_array not available
                    print(f"Warning: Could not create stereo audio for {duration}s, using mono")
                    # Keep mono audio instead
            
            # Store audio info
            test_audio[(duration, channels)] = {
                'audio': audio,
                'size_mb': get_audio_size_mb(audio),
                'samples_per_channel': int(duration * SAMPLE_RATE)
            }
            
            print(f"  {duration}s, {channels}ch: {get_audio_size_mb(audio):.2f} MB")
    
    return test_audio

# Generate all test audio
test_audio = generate_test_audio()
print(f"\nGenerated {len(test_audio)} test audio samples")

# Test what tensor methods are actually available
print("\nTesting available tensor methods...")
sample_audio = aus.generation.sine_wave(440.0, 1.0, sample_rate=44100)
print(f"to_numpy: {'✓' if hasattr(sample_audio, 'to_numpy') else '✗'}")
print(f"__array__: {'✓' if hasattr(sample_audio, '__array__') else '✗'}")
print(f"__array_ufunc__: {'✓' if hasattr(sample_audio, '__array_ufunc__') else '✗'}")
print(f"__array_function__: {'✓' if hasattr(sample_audio, '__array_function__') else '✗'}")
print(f"to_torch: {'✓' if hasattr(sample_audio, 'to_torch') else '✗'}")
print(f"__dlpack__: {'✓' if hasattr(sample_audio, '__dlpack__') else '✗'}")

# Test actual functionality
print("\nTesting actual functionality...")
try:
    arr = sample_audio.to_numpy()
    print("✓ to_numpy() works")
except Exception as e:
    print(f"✗ to_numpy() failed: {e}")

try:
    arr = sample_audio.__array__(None)
    print("✓ __array__() works")
except Exception as e:
    print(f"✗ __array__() failed: {e}")

try:
    arr = np.array(sample_audio)
    print("✓ np.array() works")
except Exception as e:
    print(f"✗ np.array() failed: {e}")

try:
    result = np.multiply(sample_audio, 0.5)
    print("✓ np.multiply() works")
except Exception as e:
    print(f"✗ np.multiply() failed: {e}")

# Test workaround approach
try:
    arr = sample_audio.to_numpy()
    result = np.multiply(arr, 0.5)
    print("✓ np.multiply(audio.to_numpy(), 0.5) works (WORKAROUND)")
except Exception as e:
    print(f"✗ workaround failed: {e}")

del sample_audio

## Benchmarking Framework

In [ ]:
def benchmark_operation(operation, iterations=BENCHMARK_ITERATIONS):
    """Benchmark an operation with multiple iterations."""
    times = []

    for i in range(iterations):
        # Ensure clean state between iterations
        gc.collect()
        
        try:
            start_time = time.perf_counter()
            result = operation()
            end_time = time.perf_counter()
            
            times.append(end_time - start_time)
            
            # Clean up result to avoid memory accumulation
            del result
            gc.collect()
            
        except Exception as e:
            print(f"Error in benchmark iteration {i}: {e}")
            continue

    if not times:
        return {
            'mean_time': 0.0,
            'std_time': 0.0,
            'min_time': 0.0,
            'max_time': 0.0
        }

    return {
        'mean_time': np.mean(times),
        'std_time': np.std(times),
        'min_time': np.min(times),
        'max_time': np.max(times)
    }

## NumPy Array Protocol Benchmarks

In [ ]:
def benchmark_numpy_protocols():
    """Benchmark NumPy array protocol methods."""
    results = []
    
    print("Benchmarking NumPy array protocol methods...")
    
    for duration in DURATIONS:
        for channels in CHANNELS:
            audio_info = test_audio[(duration, channels)]
            audio = audio_info['audio']
            size_mb = audio_info['size_mb']
            
            print(f"  Testing {duration}s, {channels}ch...")
            
            # Benchmark existing to_numpy (baseline)
            def test_to_numpy():
                return audio.to_numpy()
            
            to_numpy_stats = benchmark_operation(test_to_numpy)
            to_numpy_throughput = calculate_throughput(size_mb, to_numpy_stats['mean_time'])
            
            # Benchmark __array__ protocol method directly
            def test_array_method():
                return audio.__array__(None)
            
            try:
                array_method_stats = benchmark_operation(test_array_method)
                array_method_throughput = calculate_throughput(size_mb, array_method_stats['mean_time'])
            except Exception as e:
                print(f"    __array__ method failed: {e}")
                array_method_stats = {'mean_time': 0.0, 'std_time': 0.0}
                array_method_throughput = 0.0
            
            # Benchmark workaround: convert to numpy then operate
            def test_numpy_multiply_workaround():
                arr = audio.to_numpy()
                return np.multiply(arr, 0.5)
            
            try:
                numpy_multiply_stats = benchmark_operation(test_numpy_multiply_workaround)
                numpy_multiply_throughput = calculate_throughput(size_mb, numpy_multiply_stats['mean_time'])
            except Exception as e:
                print(f"    numpy multiply workaround failed: {e}")
                numpy_multiply_stats = {'mean_time': 0.0, 'std_time': 0.0}
                numpy_multiply_throughput = 0.0
            
            # Benchmark statistical operations with workaround
            def test_numpy_mean_workaround():
                arr = audio.to_numpy()
                return np.mean(arr)
            
            try:
                numpy_mean_stats = benchmark_operation(test_numpy_mean_workaround)
                numpy_mean_throughput = calculate_throughput(size_mb, numpy_mean_stats['mean_time'])
            except Exception as e:
                print(f"    numpy mean workaround failed: {e}")
                numpy_mean_stats = {'mean_time': 0.0, 'std_time': 0.0}
                numpy_mean_throughput = 0.0
            
            # Store results
            results.extend([
                {
                    'method': 'to_numpy',
                    'category': 'numpy_protocol',
                    'duration': duration,
                    'channels': channels,
                    'size_mb': size_mb,
                    'mean_time': to_numpy_stats['mean_time'],
                    'std_time': to_numpy_stats['std_time'],
                    'throughput_mb_s': to_numpy_throughput
                },
                {
                    'method': '__array__ method',
                    'category': 'numpy_protocol',
                    'duration': duration,
                    'channels': channels,
                    'size_mb': size_mb,
                    'mean_time': array_method_stats['mean_time'],
                    'std_time': array_method_stats['std_time'],
                    'throughput_mb_s': array_method_throughput
                },
                {
                    'method': 'np.multiply(to_numpy)',
                    'category': 'numpy_protocol',
                    'duration': duration,
                    'channels': channels,
                    'size_mb': size_mb,
                    'mean_time': numpy_multiply_stats['mean_time'],
                    'std_time': numpy_multiply_stats['std_time'],
                    'throughput_mb_s': numpy_multiply_throughput
                },
                {
                    'method': 'np.mean(to_numpy)',
                    'category': 'numpy_protocol',
                    'duration': duration,
                    'channels': channels,
                    'size_mb': size_mb,
                    'mean_time': numpy_mean_stats['mean_time'],
                    'std_time': numpy_mean_stats['std_time'],
                    'throughput_mb_s': numpy_mean_throughput
                }
            ])
            
            # Show progress
            print(f"    to_numpy:              {format_time(to_numpy_stats['mean_time'])}, {to_numpy_throughput:.1f} MB/s")
            print(f"    __array__ method:      {format_time(array_method_stats['mean_time'])}, {array_method_throughput:.1f} MB/s")
            print(f"    np.multiply(to_numpy): {format_time(numpy_multiply_stats['mean_time'])}, {numpy_multiply_throughput:.1f} MB/s")
            print(f"    np.mean(to_numpy):     {format_time(numpy_mean_stats['mean_time'])}, {numpy_mean_throughput:.1f} MB/s")
            print()
    
    return results

# Run NumPy protocol benchmarks
numpy_results = benchmark_numpy_protocols()

## DLPack Protocol Benchmarks

In [17]:
def benchmark_dlpack_protocol():
    """Benchmark DLPack protocol methods."""
    results = []
    
    print("Benchmarking DLPack protocol methods...")
    
    # Check if DLPack methods are available
    sample_audio = aus.generation.sine_wave(440.0, 0.1, sample_rate=44100)
    has_dlpack = hasattr(sample_audio, '__dlpack__')
    has_dlpack_device = hasattr(sample_audio, '__dlpack_device__')
    has_from_dlpack = hasattr(aus.AudioSamples, 'from_dlpack')
    
    if not has_dlpack:
        print("  DLPack methods not available - skipping DLPack benchmarks")
        return results
    
    for duration in DURATIONS:
        for channels in CHANNELS:
            audio_info = test_audio[(duration, channels)]
            audio = audio_info['audio']
            size_mb = audio_info['size_mb']
            
            print(f"  Testing {duration}s, {channels}ch...")
            
            # Benchmark __dlpack_device__ if available
            if has_dlpack_device:
                def test_dlpack_device():
                    return audio.__dlpack_device__()
                
                try:
                    dlpack_device_stats = benchmark_operation(test_dlpack_device)
                    dlpack_device_throughput = calculate_throughput(size_mb, dlpack_device_stats['mean_time'])
                except Exception as e:
                    print(f"    __dlpack_device__ failed: {e}")
                    dlpack_device_stats = {'mean_time': 0.0, 'std_time': 0.0}
                    dlpack_device_throughput = 0.0
            else:
                dlpack_device_stats = {'mean_time': 0.0, 'std_time': 0.0}
                dlpack_device_throughput = 0.0
            
            # Benchmark __dlpack__ export
            def test_dlpack_export():
                return audio.__dlpack__()
            
            try:
                dlpack_export_stats = benchmark_operation(test_dlpack_export)
                dlpack_export_throughput = calculate_throughput(size_mb, dlpack_export_stats['mean_time'])
            except Exception as e:
                print(f"    __dlpack__ failed: {e}")
                dlpack_export_stats = {'mean_time': 0.0, 'std_time': 0.0}
                dlpack_export_throughput = 0.0
            
            # Benchmark from_dlpack (if available and export worked)
            if has_from_dlpack and dlpack_export_stats['mean_time'] > 0:
                try:
                    # Get a dlpack tensor for testing
                    dlpack_tensor = audio.__dlpack__()
                    
                    def test_from_dlpack():
                        return aus.AudioSamples.from_dlpack(dlpack_tensor, SAMPLE_RATE)
                    
                    # Note: We can only test this once per dlpack tensor
                    # So we'll do a simpler benchmark
                    start_time = time.perf_counter()
                    restored_audio = aus.AudioSamples.from_dlpack(dlpack_tensor, SAMPLE_RATE)
                    end_time = time.perf_counter()
                    
                    from_dlpack_time = end_time - start_time
                    from_dlpack_throughput = calculate_throughput(size_mb, from_dlpack_time)
                    
                    from_dlpack_stats = {
                        'mean_time': from_dlpack_time,
                        'std_time': 0.0  # Single measurement
                    }
                except Exception as e:
                    print(f"    from_dlpack failed: {e}")
                    from_dlpack_stats = {'mean_time': 0.0, 'std_time': 0.0}
                    from_dlpack_throughput = 0.0
            else:
                from_dlpack_stats = {'mean_time': 0.0, 'std_time': 0.0}
                from_dlpack_throughput = 0.0
            
            # Store results
            if has_dlpack_device and dlpack_device_stats['mean_time'] > 0:
                results.append({
                    'method': '__dlpack_device__',
                    'category': 'dlpack',
                    'duration': duration,
                    'channels': channels,
                    'size_mb': size_mb,
                    'mean_time': dlpack_device_stats['mean_time'],
                    'std_time': dlpack_device_stats['std_time'],
                    'throughput_mb_s': dlpack_device_throughput
                })
            
            if dlpack_export_stats['mean_time'] > 0:
                results.append({
                    'method': '__dlpack__',
                    'category': 'dlpack',
                    'duration': duration,
                    'channels': channels,
                    'size_mb': size_mb,
                    'mean_time': dlpack_export_stats['mean_time'],
                    'std_time': dlpack_export_stats['std_time'],
                    'throughput_mb_s': dlpack_export_throughput
                })
            
            if from_dlpack_stats['mean_time'] > 0:
                results.append({
                    'method': 'from_dlpack',
                    'category': 'dlpack',
                    'duration': duration,
                    'channels': channels,
                    'size_mb': size_mb,
                    'mean_time': from_dlpack_stats['mean_time'],
                    'std_time': from_dlpack_stats['std_time'],
                    'throughput_mb_s': from_dlpack_throughput
                })
            
            # Show progress
            if has_dlpack_device:
                print(f"    __dlpack_device__: {format_time(dlpack_device_stats['mean_time'])}, {dlpack_device_throughput:.1f} MB/s")
            print(f"    __dlpack__:        {format_time(dlpack_export_stats['mean_time'])}, {dlpack_export_throughput:.1f} MB/s")
            print(f"    from_dlpack:       {format_time(from_dlpack_stats['mean_time'])}, {from_dlpack_throughput:.1f} MB/s")
            print()
    
    return results

# Run DLPack protocol benchmarks
dlpack_results = benchmark_dlpack_protocol()

Benchmarking DLPack protocol methods...
  DLPack methods not available - skipping DLPack benchmarks


## PyTorch Integration Benchmarks

In [ ]:
def benchmark_pytorch_integration():
    """Benchmark PyTorch integration methods."""
    results = []
    
    if not TORCH_AVAILABLE:
        print("PyTorch not available - skipping PyTorch benchmarks")
        return results
    
    print("Benchmarking PyTorch integration methods...")
    
    # Check if PyTorch methods are available
    sample_audio = aus.generation.sine_wave(440.0, 0.1, sample_rate=44100)
    has_to_torch = hasattr(sample_audio, 'to_torch')
    has_from_torch = hasattr(aus.AudioSamples, 'from_torch')
    
    if not has_to_torch:
        print("  PyTorch methods not available - testing workaround approach")
    
    for duration in DURATIONS:
        for channels in CHANNELS:
            audio_info = test_audio[(duration, channels)]
            audio = audio_info['audio']
            size_mb = audio_info['size_mb']
            
            print(f"  Testing {duration}s, {channels}ch...")
            
            # Benchmark to_torch if available
            if has_to_torch:
                def test_to_torch():
                    return audio.to_torch()
                
                try:
                    to_torch_stats = benchmark_operation(test_to_torch)
                    to_torch_throughput = calculate_throughput(size_mb, to_torch_stats['mean_time'])
                except Exception as e:
                    print(f"    to_torch failed: {e}")
                    to_torch_stats = {'mean_time': 0.0, 'std_time': 0.0}
                    to_torch_throughput = 0.0
            else:
                to_torch_stats = {'mean_time': 0.0, 'std_time': 0.0}
                to_torch_throughput = 0.0
            
            # Benchmark torch.from_numpy(audio.to_numpy()) as workaround/baseline
            def test_torch_from_numpy():
                np_array = audio.to_numpy()
                return torch.from_numpy(np_array)
            
            try:
                torch_numpy_stats = benchmark_operation(test_torch_from_numpy)
                torch_numpy_throughput = calculate_throughput(size_mb, torch_numpy_stats['mean_time'])
            except Exception as e:
                print(f"    torch.from_numpy failed: {e}")
                torch_numpy_stats = {'mean_time': 0.0, 'std_time': 0.0}
                torch_numpy_throughput = 0.0
            
            # Benchmark numpy->torch->back conversion workflow
            def test_torch_roundtrip_via_numpy():
                np_array = audio.to_numpy()
                torch_tensor = torch.from_numpy(np_array)
                return torch_tensor.numpy()
            
            try:
                torch_roundtrip_stats = benchmark_operation(test_torch_roundtrip_via_numpy)
                torch_roundtrip_throughput = calculate_throughput(size_mb, torch_roundtrip_stats['mean_time'])
            except Exception as e:
                print(f"    torch roundtrip failed: {e}")
                torch_roundtrip_stats = {'mean_time': 0.0, 'std_time': 0.0}
                torch_roundtrip_throughput = 0.0
            
            # Benchmark from_torch (if available and to_torch worked)
            if has_from_torch and has_to_torch and to_torch_stats['mean_time'] > 0:
                try:
                    # Get a torch tensor for testing
                    torch_tensor = audio.to_torch()
                    
                    def test_from_torch():
                        return aus.AudioSamples.from_torch(torch_tensor, SAMPLE_RATE)
                    
                    from_torch_stats = benchmark_operation(test_from_torch)
                    from_torch_throughput = calculate_throughput(size_mb, from_torch_stats['mean_time'])
                except Exception as e:
                    print(f"    from_torch failed: {e}")
                    from_torch_stats = {'mean_time': 0.0, 'std_time': 0.0}
                    from_torch_throughput = 0.0
            elif has_from_torch and torch_numpy_stats['mean_time'] > 0:
                # Test from_torch using the numpy->torch workaround
                try:
                    # Create torch tensor via numpy
                    np_array = audio.to_numpy()
                    torch_tensor = torch.from_numpy(np_array.copy())  # Copy to avoid shared memory issues
                    
                    def test_from_torch_via_numpy():
                        return aus.AudioSamples.from_torch(torch_tensor, SAMPLE_RATE)
                    
                    from_torch_stats = benchmark_operation(test_from_torch_via_numpy)
                    from_torch_throughput = calculate_throughput(size_mb, from_torch_stats['mean_time'])
                except Exception as e:
                    print(f"    from_torch via numpy failed: {e}")
                    from_torch_stats = {'mean_time': 0.0, 'std_time': 0.0}
                    from_torch_throughput = 0.0
            else:
                from_torch_stats = {'mean_time': 0.0, 'std_time': 0.0}
                from_torch_throughput = 0.0
            
            # Store results (only for methods that actually worked)
            if has_to_torch and to_torch_stats['mean_time'] > 0:
                results.append({
                    'method': 'to_torch',
                    'category': 'pytorch',
                    'duration': duration,
                    'channels': channels,
                    'size_mb': size_mb,
                    'mean_time': to_torch_stats['mean_time'],
                    'std_time': to_torch_stats['std_time'],
                    'throughput_mb_s': to_torch_throughput
                })
            
            if torch_numpy_stats['mean_time'] > 0:
                results.append({
                    'method': 'torch.from_numpy',
                    'category': 'pytorch',
                    'duration': duration,
                    'channels': channels,
                    'size_mb': size_mb,
                    'mean_time': torch_numpy_stats['mean_time'],
                    'std_time': torch_numpy_stats['std_time'],
                    'throughput_mb_s': torch_numpy_throughput
                })
            
            if torch_roundtrip_stats['mean_time'] > 0:
                results.append({
                    'method': 'torch_roundtrip_numpy',
                    'category': 'pytorch',
                    'duration': duration,
                    'channels': channels,
                    'size_mb': size_mb,
                    'mean_time': torch_roundtrip_stats['mean_time'],
                    'std_time': torch_roundtrip_stats['std_time'],
                    'throughput_mb_s': torch_roundtrip_throughput
                })
            
            if has_from_torch and from_torch_stats['mean_time'] > 0:
                results.append({
                    'method': 'from_torch',
                    'category': 'pytorch',
                    'duration': duration,
                    'channels': channels,
                    'size_mb': size_mb,
                    'mean_time': from_torch_stats['mean_time'],
                    'std_time': from_torch_stats['std_time'],
                    'throughput_mb_s': from_torch_throughput
                })
            
            # Show progress
            if has_to_torch:
                print(f"    to_torch:          {format_time(to_torch_stats['mean_time'])}, {to_torch_throughput:.1f} MB/s")
            print(f"    torch.from_numpy:  {format_time(torch_numpy_stats['mean_time'])}, {torch_numpy_throughput:.1f} MB/s")
            print(f"    torch roundtrip:   {format_time(torch_roundtrip_stats['mean_time'])}, {torch_roundtrip_throughput:.1f} MB/s")
            if has_from_torch:
                print(f"    from_torch:        {format_time(from_torch_stats['mean_time'])}, {from_torch_throughput:.1f} MB/s")
            print()
    
    return results

# Run PyTorch integration benchmarks
pytorch_results = benchmark_pytorch_integration()

## Results Analysis

In [ ]:
# Combine all results
all_results = numpy_results + dlpack_results + pytorch_results
df = pd.DataFrame(all_results)

print("Tensor Interoperability Benchmark Results Summary:")
print("=" * 60)

# Overall statistics by category
for category in df['category'].unique():
    print(f"\n{category.upper()} Methods:")
    cat_data = df[df['category'] == category]
    
    for method in cat_data['method'].unique():
        method_data = cat_data[cat_data['method'] == method]
        if len(method_data) > 0:
            avg_time = method_data['mean_time'].mean()
            avg_throughput = method_data['throughput_mb_s'].mean()
            
            print(f"  {method:16}: {format_time(avg_time):>8} avg, {avg_throughput:>6.1f} MB/s avg")

# Save results
results_file = os.path.join(temp_dir, "tensor_benchmark_results.csv")
df.to_csv(results_file, index=False)
print(f"\nResults saved to: {results_file}")
print(f"Total benchmarks: {len(all_results)}")

## Performance Visualizations

In [ ]:
# Create output directory for figures
fig_dir = os.path.join(temp_dir, "figures")
os.makedirs(fig_dir, exist_ok=True)

def save_figure(fig, name, dpi=300):
    """Save figure as both PDF and PNG with high quality."""
    pdf_path = os.path.join(fig_dir, f"{name}.pdf")
    png_path = os.path.join(fig_dir, f"{name}.png")
    
    fig.savefig(pdf_path, format='pdf', dpi=dpi, bbox_inches='tight', 
               facecolor='white', edgecolor='none')
    fig.savefig(png_path, format='png', dpi=dpi, bbox_inches='tight',
               facecolor='white', edgecolor='none')
    
    print(f"Saved: {pdf_path}")
    print(f"Saved: {png_path}")

# Define consistent colors and markers
marker_map = {1: 'o', 2: 's'}  # circle for mono, square for stereo

def plot_category_performance(df, category, title):
    """Plot performance for a specific category."""
    fig, ax = plt.subplots(figsize=(12, 9))
    
    cat_data = df[df['category'] == category]
    methods = cat_data['method'].unique()
    
    for i, method in enumerate(methods):
        for channels in CHANNELS:
            subset = cat_data[(cat_data['method'] == method) & (cat_data['channels'] == channels)]
            if len(subset) > 0:
                channel_name = 'mono' if channels == 1 else 'stereo'
                ax.plot(subset['duration'], subset['mean_time'],
                       marker=marker_map[channels], label=f"{method} ({channel_name})",
                       color=OKABE_ITO_COLORS[i], linewidth=4, markersize=12)

    ax.set_xlabel('Audio Duration (seconds)', fontweight='bold')
    ax.set_ylabel('Time (seconds)', fontweight='bold')
    ax.set_title(title, fontweight='bold')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.grid(True, alpha=0.3)
    
    legend = ax.legend(bbox_to_anchor=(0.5, -0.15), loc='upper center', ncol=2)
    legend.get_frame().set_facecolor('white')
    legend.get_frame().set_alpha(0.9)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)
    return fig

# Generate plots for each category
if len(numpy_results) > 0:
    numpy_fig = plot_category_performance(df, 'numpy_protocol', 'NumPy Array Protocol Performance')
    save_figure(numpy_fig, "numpy_protocol_performance")
    plt.show()

if len(dlpack_results) > 0:
    dlpack_fig = plot_category_performance(df, 'dlpack', 'DLPack Protocol Performance')
    save_figure(dlpack_fig, "dlpack_protocol_performance")
    plt.show()

if len(pytorch_results) > 0:
    pytorch_fig = plot_category_performance(df, 'pytorch', 'PyTorch Integration Performance')
    save_figure(pytorch_fig, "pytorch_integration_performance")
    plt.show()

In [ ]:
# Throughput comparison
if len(df) > 0:
    fig, ax = plt.subplots(figsize=(15, 9))
    
    # Calculate average throughput by method
    throughput_data = df.groupby(['category', 'method'])['throughput_mb_s'].mean().reset_index()
    
    # Create combined labels
    throughput_data['method_label'] = throughput_data['category'] + '\n' + throughput_data['method']
    
    # Plot
    bars = ax.bar(range(len(throughput_data)), throughput_data['throughput_mb_s'], 
                  color=OKABE_ITO_COLORS[:len(throughput_data)])
    
    ax.set_xlabel('Method', fontweight='bold')
    ax.set_ylabel('Average Throughput (MB/s)', fontweight='bold')
    ax.set_title('Tensor Interoperability Throughput Comparison', fontweight='bold')
    ax.set_xticks(range(len(throughput_data)))
    ax.set_xticklabels(throughput_data['method_label'], rotation=45, ha='right')
    ax.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for i, (bar, value) in enumerate(zip(bars, throughput_data['throughput_mb_s'])):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
               f'{value:.1f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    save_figure(fig, "throughput_comparison")
    plt.show()

print(f"\nAll figures saved to: {fig_dir}")

## Summary and Conclusions

In [ ]:
# Generate final summary report
print("\n" + "="*70)
print("TENSOR INTEROPERABILITY BENCHMARK SUMMARY")
print("="*70)

if len(df) > 0:
    # Find fastest methods in each category
    print("\nFastest Methods by Category:")
    for category in df['category'].unique():
        cat_data = df[df['category'] == category]
        fastest = cat_data.loc[cat_data['throughput_mb_s'].idxmax()]
        print(f"  {category}: {fastest['method']} ({fastest['throughput_mb_s']:.1f} MB/s)")
    
    # Overall performance insights
    print("\nPerformance Insights:")
    
    # NumPy protocol comparison
    numpy_data = df[df['category'] == 'numpy_protocol']
    if len(numpy_data) > 0:
        to_numpy_avg = numpy_data[numpy_data['method'] == 'to_numpy']['throughput_mb_s'].mean()
        array_method_avg = numpy_data[numpy_data['method'] == '__array__ method']['throughput_mb_s'].mean()
        
        if array_method_avg > 0 and to_numpy_avg > 0:
            ratio = to_numpy_avg / array_method_avg
            faster_method = "to_numpy" if ratio > 1 else "__array__ method"
            print(f"• {faster_method} is {max(ratio, 1/ratio):.2f}x faster than {'__array__ method' if ratio > 1 else 'to_numpy'}")
        
        # Check if workaround operations are efficient
        multiply_avg = numpy_data[numpy_data['method'] == 'np.multiply(to_numpy)']['throughput_mb_s'].mean()
        if multiply_avg > 0 and to_numpy_avg > 0:
            ratio = multiply_avg / to_numpy_avg
            print(f"• NumPy operations via to_numpy() add {(1-ratio)*100:.1f}% overhead")
    
    # PyTorch comparison
    torch_data = df[df['category'] == 'pytorch']
    if len(torch_data) > 0:
        torch_numpy_avg = torch_data[torch_data['method'] == 'torch.from_numpy']['throughput_mb_s'].mean()
        roundtrip_avg = torch_data[torch_data['method'] == 'torch_roundtrip_numpy']['throughput_mb_s'].mean()
        
        if torch_numpy_avg > 0 and roundtrip_avg > 0:
            ratio = torch_numpy_avg / roundtrip_avg  
            print(f"• torch.from_numpy is {ratio:.2f}x faster than full roundtrip conversion")
        
        # Check if native to_torch exists
        to_torch_data = torch_data[torch_data['method'] == 'to_torch']
        if len(to_torch_data) > 0:
            to_torch_avg = to_torch_data['throughput_mb_s'].mean()
            if torch_numpy_avg > 0:
                ratio = to_torch_avg / torch_numpy_avg
                faster_method = "to_torch" if ratio > 1 else "torch.from_numpy"
                print(f"• {faster_method} is {max(ratio, 1/ratio):.2f}x faster than {'torch.from_numpy' if ratio > 1 else 'to_torch'}")
        else:
            print("• Native to_torch method not available - use torch.from_numpy(audio.to_numpy()) workaround")
    
    # DLPack assessment
    dlpack_data = df[df['category'] == 'dlpack']
    if len(dlpack_data) == 0:
        print("• DLPack methods not available - feature not implemented")
else:
    print("\nNo benchmark results available for analysis.")

print(f"\nTest Configuration:")
print(f"  Sample Rate: {SAMPLE_RATE} Hz")
print(f"  Duration Range: {min(DURATIONS)}-{max(DURATIONS)} seconds")
print(f"  Iterations per test: {BENCHMARK_ITERATIONS}")
print(f"  Total benchmarks: {len(all_results)}")

# Implementation status summary
print(f"\nImplementation Status:")
sample_audio = aus.generation.sine_wave(440.0, 0.1, sample_rate=44100)
print(f"  ✓ to_numpy() - Core NumPy conversion")
print(f"  ✓ __array__() method - Array protocol support")
print(f"  {'✓' if hasattr(sample_audio, '__array_ufunc__') else '✗'} __array_ufunc__ - Direct NumPy ufunc support")
print(f"  {'✓' if hasattr(sample_audio, '__array_function__') else '✗'} __array_function__ - Direct NumPy function support")
print(f"  {'✓' if hasattr(sample_audio, 'to_torch') else '✗'} to_torch() - Native PyTorch conversion")
print(f"  {'✓' if hasattr(aus.AudioSamples, 'from_torch') else '✗'} from_torch() - PyTorch to AudioSamples")
print(f"  {'✓' if hasattr(sample_audio, '__dlpack__') else '✗'} DLPack protocol - Zero-copy tensor exchange")

print(f"\nRecommended Usage:")
print(f"  • For NumPy operations: use audio.to_numpy() then standard NumPy functions")
print(f"  • For PyTorch operations: use torch.from_numpy(audio.to_numpy())")
print(f"  • Performance is excellent for these workaround approaches")

print("\nBenchmark complete!")
print(f"Results and figures saved to: {temp_dir}")